In [1]:
# установка необходимых библиотек с использованием файла requirements.txt, опубликованного в GitHub репозитории проекта

!pip install -r https://raw.githubusercontent.com/ponomarchuk-anna/Thesis/refs/heads/main/Gradio%20App/requirements.txt

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-install-86_8zutl/segment-anything_45d49511d1474374bf90239a15f9bcd3
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-install-86_8zutl/segment-anything_45d49511d1474374bf90239a15f9bcd3
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# импорт библиотек, которые понадобятся в дальнейшем

import cv2
import gradio as gr
import numpy as np
import torch
import torch.nn.functional as F

from PIL import Image, ImageDraw
from pathlib import Path
from segment_anything import sam_model_registry

In [3]:
# предварительно финальные веса обученных моделей были загружены на гугл диск

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# для каждой из моделей загружаем веса, полученные в процессе выполнения ВКР

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_model(path_to_weights):
    # инициализируем модель, используя чекпоинт
    model_type = 'vit_b'
    model = sam_model_registry[model_type](checkpoint=str(path_to_weights))
    model.to(device)

    # переводим модель в режим оценки
    model.eval()
    return model

In [5]:
# загружаем веса для всех моделей, они хранятся в root_directory

root_directory = Path('/content/drive/MyDrive/Thesis/SAM fine-tuning/weights')

# сохраним список из кортежей, первый элемент которых - название файла с весами модели,
# второй - соответствующая модели метка, которая будет указана в веб-приложении
all_models = [
    ('wound.pth', 'Вся рана'),
    ('pigmentation.pth', 'Вторичная пигментация (модель недообучена, не рекомендуется использовать)'),
    ('hyperemia.pth', 'Зона гиперемии вокруг (модель недообучена, не рекомендуется использовать)'),
    ('granulations.pth', 'Зона грануляций'),
    ('necrosis.pth', 'Зона некроза (модель недообучена, не рекомендуется использовать)'),
    ('edema.pth', 'Зона отёка вокруг раны (модель недообучена, не рекомендуется использовать)'),
    ('stitches.pth', 'Зона шва'),
    ('metal.pth', 'Металлоконструкция (модель недообучена, не рекомендуется использовать)'),
    ('subcutaneous_fat.pth', 'Подкожная жировая клетчатка без грануляций'),
    ('fascia.pth', 'Фасция без грануляций'),
    ('fibrin.pth', 'Фибрин')
]

# в словаре будем хранить модели для каждого из классов
models = {}

for weights_name, label in all_models:
    models[label] = get_model(root_directory / Path(weights_name))

In [6]:
# функция сегментирует загруженную фотографию с использованием указанных пользователем моделей

def get_result(image, select_all, models_used, threshold):
    '''
    image - загруженное пользователем медицинское изображение, которое необходимо сегментировать
    select_all - флаг, означающий, что на изображении нужно выделить все классы
    models_used - список моделей в зависимости от того, какие классы пользователь хочет выделить на фотографии
    threshold - порог бинаризации для создания бинарных масок
    '''

    masks = []

    if select_all:
        models_used = list(models.keys())

    # преобразуем изображение в формат RGB, приведём его к требуемому размеру 1024 на 1024, а затем превратим в тензор
    tensor_image = image.convert('RGB').resize((1024, 1024))
    tensor_image = torch.from_numpy(np.array(tensor_image)).permute(2, 0, 1).float() / 255.0
    tensor_image = tensor_image.unsqueeze(0).to(device)

    height, width = tensor_image.shape[2], tensor_image.shape[3]
    bounding_box = torch.tensor([[0, 0, width, height]], device=device, dtype=torch.float)

    # запускаем каждую из выбранных пользователем моделей
    for current_model in models_used:
        model = models[current_model]

        with torch.no_grad():
            # пропускаем фотографию через энкодер, а также получаем позиционные эмбеддинги
            image_embedding = model.image_encoder(tensor_image)
            positional_embedding = model.prompt_encoder.get_dense_pe()

            # передаём модели только bounding_box, найденный ранее
            sparse_prompt, dense_prompt = model.prompt_encoder(
                points=None,
                boxes=bounding_box,
                masks=None
            )

            # декодируем маску
            predicted_logits, _ = model.mask_decoder(
                image_embeddings=image_embedding,
                image_pe=positional_embedding,
                sparse_prompt_embeddings=sparse_prompt,
                dense_prompt_embeddings=dense_prompt,
                multimask_output=False
            )

        # получаем предсказанную моделью бинарную маску
        predicted_mask = (torch.sigmoid(predicted_logits) >= threshold).float()
        predicted_mask_resized = F.interpolate(predicted_mask, size=(image.size[1], image.size[0]), mode='nearest').squeeze().cpu().numpy().astype(np.uint8)

        # для комфортного восприятия маски пользователем найдём контуры предсказанной маски
        contour, _ = cv2.findContours(predicted_mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        # создаём холст для отображения полученных результатов
        canvas = Image.new('RGBA', image.size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(canvas)
        fill_colour = (128, 0, 32, 100)
        contour_colour = (128, 0, 32, 255)

        # пройдёмся по каждому из найденных контуров для красивой отрисовки
        for current_contour in contour:
            points = current_contour.squeeze(1)
            coordinates = [(int(x), int(y)) for x, y in points]

            # для отрисовки многоугольника требуется не менее трёх точек
            if len(coordinates) > 2:
                draw.polygon(coordinates, fill=fill_colour, outline=contour_colour)
                draw.line(coordinates + [coordinates[0]], fill=contour_colour, width=3)

        # теперь накладываем полученную маску на исходное изображение и сохраняем результат
        result = Image.alpha_composite(image.convert('RGBA'), canvas)
        masks.append((result, current_model))
    return masks

In [ ]:
def build_interface():
    description ='### Предупреждаем, что в данный момент полученные результаты не могут считаться точными и использоваться для принятия медицинских решений ###'
    demo = gr.Interface(
        fn=get_result,
        inputs=[
            gr.Image(type='pil', label='Исходное изображение'),
            gr.Checkbox(label='Выбрать все зоны', value=False),
            gr.CheckboxGroup(
                choices=list(models.keys()),
                label='Зоны, которые необходимо выделить на фотографии'
            ),
            gr.Slider(minimum=0.0, maximum=1.0, step=0.01, value=0.5, label='Требуемый порог бинаризации'),
        ],
        outputs=gr.Gallery(label='Результаты сегментации', columns=2, height='auto'),
        title='Демонстрация работы моделей для выделения раны и сегментации тканей внутри неё',
        description=description,
        allow_flagging='never'
    )
    return demo

if __name__ == "__main__":
    demo = build_interface()
    demo.launch(share=True, debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6a6b373ffcfc5c97af.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
